# Graphical user interfaces
In this notebook we ask bob to create a user interface that allows interacting with a plot.

In [1]:
import os
from bia_bob import bob
bob.initialize(endpoint='https://chat-ai.academiccloud.de/v1', model='openai-gpt-oss-120b', api_key=os.environ.get('KISSKI_API_KEY'))

In this example we use a [videogame sales dataset by Gregory Smith](https://zenodo.org/records/5898311) which is licensed [CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/legalcode).

In [2]:
%%bob
write python code that
* opens the file vgsales.csv using pandas
* if the file doesn't exist, open it from https://zenodo.org/records/5898311/files/vgsales.csv?download=1 
  and save it as vgsales.csv

The script loads `vgsales.csv` with pandas.  
If the file is missing, it downloads the CSV from the Zenodo URL, saves it locally as `vgsales.csv`, and then loads it.



In [ ]:
import os
import pandas as pd
import urllib.request

# 1. Define file name and remote URL
local_file = "vgsales.csv"
remote_url = (
    "https://zenodo.org/records/5898311/files/vgsales.csv?download=1"
)

# 2. Download if missing
if not os.path.exists(local_file):
    print(f"File '{local_file}' not found. Downloading from Zenodo...")
    urllib.request.urlretrieve(remote_url, local_file)
    print("Download complete.")

# 3. Load into pandas DataFrame
df = pd.read_csv(local_file)

# Optional: show the first few rows
print(df.head())

In [4]:
%%bob
Build a graphical user interface using ipywidgets where the user can select the publisher. After the publisher is selected, draw a bar plot showing the NA_Sales, EU_Sales, JP_Sales and Other_Sales.
The bar-plot should be shown right below the pulldown

The code creates an interactive widget that lets the user pick a **publisher** from the *vgsales* dataset.  
When a publisher is chosen, a bar chart is displayed directly below the drop‑down, showing the total (or summed) sales of that publisher in **NA**, **EU**, **JP** and **Other** regions.



In [ ]:
# ------------------------------------------------------------
# 1. Imports & data loading (reuse df if already defined)
# ------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display

# Ensure the CSV exists – download only if missing
if not os.path.exists("vgsales.csv"):
    urllib.request.urlretrieve(
        "https://zenodo.org/records/5898311/files/vgsales.csv?download=1",
        "vgsales.csv"
    )

# Load the dataframe only if it hasn't been defined yet
if "df" not in globals():
    df = pd.read_csv("vgsales.csv")

# ------------------------------------------------------------
# 2. Widget creation
# ------------------------------------------------------------
# Dropdown with all unique publishers (sorted for readability)
publisher_dropdown = widgets.Dropdown(
    options=sorted(df["Publisher"].dropna().unique()),
    description="Publisher:",
    value=None,
    layout=widgets.Layout(width='300px')
)

# Output area for the bar plot
plot_out = widgets.Output()

# ------------------------------------------------------------
# 3. Callback to update the plot
# ------------------------------------------------------------
def update_plot(change):
    # Only react to actual value changes
    if change['name'] != 'value':
        return
    
    selected_pub = change['new']
    
    with plot_out:
        # Clear previous plot
        plot_out.clear_output()
        if selected_pub is None:
            return
        
        # Filter rows belonging to the selected publisher
        sub = df[df["Publisher"] == selected_pub]
        
        # Sum sales for each region (ignore NaNs)
        sales = sub[["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]].sum()
        
        # Plot
        plt.figure(figsize=(6, 4))
        sns.barplot(x=sales.index, y=sales.values, palette="viridis")
        plt.title(f"Total sales for publisher: {selected_pub}")
        plt.ylabel("Sales (million units)")
        plt.xlabel("Region")
        plt.ylim(0, max(sales.max() * 1.1, 1))  # a little headroom
        plt.show()

# Attach the observer
publisher_dropdown.observe(update_plot, names='value')

# ------------------------------------------------------------
# 4. Display the UI
# ------------------------------------------------------------
ui = widgets.VBox([publisher_dropdown, plot_out])
display(ui)

## Exercise
Extend the user interface above by an additional pulldown for selecting which column should serve as `hue` for the visualization of the data points.